# Load and install package

In [ ]:
!pip install pandas

In [1]:
import pandas as pd
import json, os, csv

# Process and Analysis

In [8]:
df_comments = pd.read_json('data/initial-data/scraping/comments_by_classes.json')
df_feedback = pd.read_json('data/initial-data/scraping/feedback_by_classes.json')
manual_collecting = pd.read_csv('data/results/engineering-1.csv')

In [9]:
df_comments

,user_statement,judul,rating,type
0,perencanaan yang disusun dalam pemerintah daer...,Kelas Online Memahami Reformasi Birokrasi di I...,,komentar/pertanyaan
1,Perlunya penguatan internalisasi RB diseluruh ASN,Kelas Online Memahami Reformasi Birokrasi di I...,,komentar/pertanyaan
2,alhamdulillah,Kelas Online Memahami Reformasi Birokrasi di I...,,komentar/pertanyaan
3,apa dampak jika dynamic governance tidak diimp...,Kelas Online Memahami Reformasi Birokrasi di I...,,komentar/pertanyaan
4,Bagaimana menerapkan Reformasi Birokrasi yang ...,Kelas Online Memahami Reformasi Birokrasi di I...,,komentar/pertanyaan
5,Paradigma administrasi apa yang saat ini ada d...,Kelas Online Memahami Reformasi Birokrasi di I...,,komentar/pertanyaan
6,seberapa efektif birokrasi tematik dalam menye...,Kelas Online Memahami Reformasi Birokrasi di I...,,komentar/pertanyaan
7,Apa dasar hukum dari materi dalam kelas pembel...,Kelas Online Memahami Reformasi Birokrasi di I...,,komentar/pertanyaan


In [11]:
df_feedback['user_statement'] = df_feedback['user_statement'].astype(str).str.strip()
df_feedback = df_feedback[~df_feedback['user_statement'].str.fullmatch(r'[-.,!? ]+')]
df_feedback

,user_statement,rating,kategori,pusat_inovasi,status,tema,judul,type
0,Kelasnya seru,5,Smartclass,Semua Pusat Inovasi,publish,Semua tema,Kelas Online Memahami Manajemen Talenta,feedback
2,OK,5,Smartclass,Semua Pusat Inovasi,publish,Semua tema,Kelas Online Memahami Manajemen Talenta,feedback
5,terbaik,5,Smartclass,Semua Pusat Inovasi,publish,Semua tema,Kelas Online Pendalaman Manajemen Pelayanan Pu...,feedback
6,good,5,Smartclass,Semua Pusat Inovasi,publish,Semua tema,Kelas Online Pendalaman Manajemen Pelayanan Pu...,feedback
7,Materi yg sangat bagus,5,Smartclass,Semua Pusat Inovasi,publish,Semua tema,Kelas Online Pendalaman Manajemen Pelayanan Pu...,feedback
...,...,...,...,...,...,...,...,...
70,good,5,Smartclass,Semua Pusat Inovasi,publish,Semua tema,Kelas Online Memahami Reformasi Birokrasi di I...,feedback
71,okaaay,5,Smartclass,Semua Pusat Inovasi,publish,Semua tema,Kelas Online Memahami Reformasi Birokrasi di I...,feedback
72,sudah bagus,4,Smartclass,Semua Pusat Inovasi,publish,Semua tema,Kelas Online Memahami Reformasi Birokrasi di I...,feedback
73,Menarik,5,Smartclass,Semua Pusat Inovasi,publish,Semua tema,Kelas Online Memahami Reformasi Birokrasi di I...,feedback


In [ ]:
new_df = pd.concat([df_feedback, df_comments], axis=0)
new_df.to_csv('data/results/scrapping-results.csv', index=False)

In [ ]:
def read_csv(filename):
    with open(filename, 'r', encoding='utf-16', errors='ignore') as f:
        reader = csv.reader(f)
        rows = list(reader)  

    header = rows[0]
    data = rows[1:]

    data_clean = [row for row in data if len(row) == len(header)]
    df = pd.DataFrame(data_clean, columns=header)
    return df

In [ ]:
review_users = os.listdir('data/initial-data/review-playstore')
df = pd.DataFrame({})

for review in review_users:
    temp = read_csv(f'data/initial-data/review-playstore/{review}')
    df = pd.concat([df, temp], axis=0)

In [ ]:
import pandas as pd

df_review = df[['Star Rating', 'Review Text']].copy()
df_review = df_review.rename(columns={'Star Rating': 'rating', 'Review Text': 'question'})
df_review['question'] = df_review['question'].astype(str).str.strip()
df_review['question'] = df_review['question'].replace('', pd.NA)
df_review = df_review.dropna(subset=['question']).reset_index(drop=True)

df_review

In [ ]:
df_join = pd.concat([df_review, new_df], axis=0)
df_join

In [ ]:
df_join.to_csv('data/results/scrapping-review.csv', index=False)